In [1]:
import os
import pandas as pd
from datetime import datetime
from sklearn.preprocessing import StandardScaler

base_dir = "../pgm"
dataframes = {}

for year in range(2016, 2022):
    year_dir = os.path.join(base_dir, f"year={year}/")
    
    if os.path.exists(year_dir):
        year_dataframes = []
        
        for file in os.listdir(year_dir):
            if file.endswith(".parquet"):
                file_path = os.path.join(year_dir, file)
                
                df = pd.read_parquet(file_path)
                
                year_dataframes.append(df)
        
        if year_dataframes:
            dataframes[year] = pd.concat(year_dataframes, ignore_index=True)

master_df = pd.concat(dataframes.values(), ignore_index=True)
master_df

,month_id,priogrid_gid,ged_sb,ged_os,ged_ns,ln_pop_gpw_sum,decay_ged_sb_1,decay_ged_sb_25,decay_ged_os_1,decay_ged_sb_5,...,treelag_1_os,treelag_2_ns,treelag_2_os,sptime_dist_k1_ged_os,sptime_dist_k1_ged_ns,sptime_dist_k10_ged_os,sptime_dist_k10_ged_ns,sptime_dist_k001_ged_os,sptime_dist_k001_ged_ns,month
0,433,62356,0.0,0.0,0.0,0.000000,3.814697e-06,3.814697e-06,3.814697e-06,1.455192e-11,...,3.985346,0.019056,0.027109,35.608988,45.915684,44.322680,54.664888,17.019060,17.497774,1.0
1,433,79599,0.0,0.0,0.0,9.145198,3.814697e-06,3.814697e-06,3.814697e-06,1.455192e-11,...,5.488608,0.031845,0.050813,29.736341,35.174565,33.049206,41.584252,2.814765,2.206377,1.0
2,433,79600,0.0,0.0,0.0,8.394584,3.814697e-06,3.814697e-06,3.814697e-06,1.455192e-11,...,5.488975,0.031858,0.050815,29.605743,35.064227,32.901368,41.743263,2.858828,2.370253,1.0
3,433,79601,0.0,0.0,0.0,9.903443,3.814697e-06,3.814697e-06,3.814697e-06,1.455192e-11,...,5.488996,0.031868,0.050813,29.483046,34.960692,32.760495,41.907637,2.987122,2.620706,1.0
4,433,80317,0.0,0.0,0.0,12.290978,3.814697e-06,3.814697e-06,3.814697e-06,1.455192e-11,...,5.535628,0.032332,0.051720,29.563491,35.014283,32.893768,40.804412,2.696757,2.148977,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1048795,502,190496,0.0,0.0,0.0,10.408626,5.199946e-07,5.199946e-07,5.199946e-07,2.703944e-13,...,4.324970,0.069880,0.058797,11.916375,10.594810,14.326549,10.977249,1.524762,10.405138,NaN
1048796,502,190507,0.0,0.0,0.0,6.647283,5.199946e-07,5.199946e-07,5.199946e-07,2.703944e-13,...,4.273955,0.092855,0.059402,8.154753,6.595453,9.899495,6.800735,2.649226,6.519210,NaN
1048797,502,190508,0.0,0.0,0.0,4.562102,5.199946e-07,5.199946e-07,5.199946e-07,2.703944e-13,...,4.265913,0.095266,0.059457,7.889867,6.344289,9.552487,6.519202,2.787185,6.264990,NaN
1048798,502,190510,0.0,0.0,0.0,7.619576,5.199946e-07,5.199946e-07,5.199946e-07,2.703944e-13,...,4.253587,0.100458,0.059811,7.433034,5.937171,8.902247,6.041523,3.281524,5.852358,NaN


In [2]:
month_id_map = {}
for year in range(1990, 2022):
    for month in range(1, 13):
        month_id = (year - 1990) * 12 + month + 120
        month_id_map[month_id] = datetime(year, month, 1)

master_df['date'] = master_df['month_id'].map(month_id_map)
master_df.drop(columns=['month'], inplace=True)
master_df

,month_id,priogrid_gid,ged_sb,ged_os,ged_ns,ln_pop_gpw_sum,decay_ged_sb_1,decay_ged_sb_25,decay_ged_os_1,decay_ged_sb_5,...,treelag_1_os,treelag_2_ns,treelag_2_os,sptime_dist_k1_ged_os,sptime_dist_k1_ged_ns,sptime_dist_k10_ged_os,sptime_dist_k10_ged_ns,sptime_dist_k001_ged_os,sptime_dist_k001_ged_ns,date
0,433,62356,0.0,0.0,0.0,0.000000,3.814697e-06,3.814697e-06,3.814697e-06,1.455192e-11,...,3.985346,0.019056,0.027109,35.608988,45.915684,44.322680,54.664888,17.019060,17.497774,2016-01-01
1,433,79599,0.0,0.0,0.0,9.145198,3.814697e-06,3.814697e-06,3.814697e-06,1.455192e-11,...,5.488608,0.031845,0.050813,29.736341,35.174565,33.049206,41.584252,2.814765,2.206377,2016-01-01
2,433,79600,0.0,0.0,0.0,8.394584,3.814697e-06,3.814697e-06,3.814697e-06,1.455192e-11,...,5.488975,0.031858,0.050815,29.605743,35.064227,32.901368,41.743263,2.858828,2.370253,2016-01-01
3,433,79601,0.0,0.0,0.0,9.903443,3.814697e-06,3.814697e-06,3.814697e-06,1.455192e-11,...,5.488996,0.031868,0.050813,29.483046,34.960692,32.760495,41.907637,2.987122,2.620706,2016-01-01
4,433,80317,0.0,0.0,0.0,12.290978,3.814697e-06,3.814697e-06,3.814697e-06,1.455192e-11,...,5.535628,0.032332,0.051720,29.563491,35.014283,32.893768,40.804412,2.696757,2.148977,2016-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1048795,502,190496,0.0,0.0,0.0,10.408626,5.199946e-07,5.199946e-07,5.199946e-07,2.703944e-13,...,4.324970,0.069880,0.058797,11.916375,10.594810,14.326549,10.977249,1.524762,10.405138,2021-10-01
1048796,502,190507,0.0,0.0,0.0,6.647283,5.199946e-07,5.199946e-07,5.199946e-07,2.703944e-13,...,4.273955,0.092855,0.059402,8.154753,6.595453,9.899495,6.800735,2.649226,6.519210,2021-10-01
1048797,502,190508,0.0,0.0,0.0,4.562102,5.199946e-07,5.199946e-07,5.199946e-07,2.703944e-13,...,4.265913,0.095266,0.059457,7.889867,6.344289,9.552487,6.519202,2.787185,6.264990,2021-10-01
1048798,502,190510,0.0,0.0,0.0,7.619576,5.199946e-07,5.199946e-07,5.199946e-07,2.703944e-13,...,4.253587,0.100458,0.059811,7.433034,5.937171,8.902247,6.041523,3.281524,5.852358,2021-10-01


In [3]:
scaler = StandardScaler()
master_df_nonscaled = master_df
master_df= pd.DataFrame(scaler.fit_transform(master_df.drop(columns=['month_id', 'priogrid_gid', 'date'])), columns=master_df.columns.drop(['month_id', 'priogrid_gid', 'date']))
master_df['month_id'] = master_df_nonscaled['month_id']
master_df['priogrid_gid'] = master_df_nonscaled['priogrid_gid']
master_df['date'] = master_df_nonscaled['date']
master_df

,ged_sb,ged_os,ged_ns,ln_pop_gpw_sum,decay_ged_sb_1,decay_ged_sb_25,decay_ged_os_1,decay_ged_sb_5,decay_ged_sb_100,decay_ged_sb_500,...,treelag_2_os,sptime_dist_k1_ged_os,sptime_dist_k1_ged_ns,sptime_dist_k10_ged_os,sptime_dist_k10_ged_ns,sptime_dist_k001_ged_os,sptime_dist_k001_ged_ns,month_id,priogrid_gid,date
0,-0.026241,-0.021117,-0.015904,-4.441878,-0.273763,-0.152357,-0.240096,-0.193102,-0.06916,-0.028508,...,-0.153469,7.579966,6.400377,6.532327,6.738254,9.639623,4.738637,433,62356,2016-01-01
1,-0.026241,-0.021117,-0.015904,-0.299347,-0.273763,-0.152357,-0.240096,-0.193102,-0.06916,-0.028508,...,-0.149642,6.073162,4.577182,4.485963,4.768209,0.392014,-0.475191,433,79599,2016-01-01
2,-0.026241,-0.021117,-0.015904,-0.639355,-0.273763,-0.152357,-0.240096,-0.193102,-0.06916,-0.028508,...,-0.149642,6.039653,4.558453,4.459127,4.792157,0.420701,-0.419315,433,79600,2016-01-01
3,-0.026241,-0.021117,-0.015904,0.044118,-0.273763,-0.152357,-0.240096,-0.193102,-0.06916,-0.028508,...,-0.149642,6.008172,4.540879,4.433556,4.816913,0.504226,-0.333919,433,79601,2016-01-01
4,-0.026241,-0.021117,-0.015904,1.125607,-0.273763,-0.152357,-0.240096,-0.193102,-0.06916,-0.028508,...,-0.149496,6.028812,4.549976,4.457748,4.650759,0.315186,-0.494762,433,80317,2016-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1048795,-0.026241,-0.021117,-0.015904,0.272952,-0.273781,-0.152391,-0.240117,-0.193102,-0.06916,-0.028508,...,-0.148353,1.500916,0.405020,1.087421,0.158557,-0.447833,2.320298,502,190496,2021-10-01
1048796,-0.026241,-0.021117,-0.015904,-1.430836,-0.273781,-0.152391,-0.240117,-0.193102,-0.06916,-0.028508,...,-0.148255,0.535759,-0.273830,0.283821,-0.470458,0.284241,0.995333,502,190507,2021-10-01
1048797,-0.026241,-0.021117,-0.015904,-2.375368,-0.273781,-0.152391,-0.240117,-0.193102,-0.06916,-0.028508,...,-0.148247,0.467794,-0.316462,0.220832,-0.512859,0.374058,0.908653,502,190508,2021-10-01
1048798,-0.026241,-0.021117,-0.015904,-0.990413,-0.273781,-0.152391,-0.240117,-0.193102,-0.06916,-0.028508,...,-0.148189,0.350580,-0.385566,0.102800,-0.584801,0.695895,0.767960,502,190510,2021-10-01


In [4]:
master_df_nonscaled = master_df_nonscaled.drop(columns=['date'])

In [5]:
master_df_nonscaled

,month_id,priogrid_gid,ged_sb,ged_os,ged_ns,ln_pop_gpw_sum,decay_ged_sb_1,decay_ged_sb_25,decay_ged_os_1,decay_ged_sb_5,...,treelag_1_ns,treelag_1_os,treelag_2_ns,treelag_2_os,sptime_dist_k1_ged_os,sptime_dist_k1_ged_ns,sptime_dist_k10_ged_os,sptime_dist_k10_ged_ns,sptime_dist_k001_ged_os,sptime_dist_k001_ged_ns
0,433,62356,0.0,0.0,0.0,0.000000,3.814697e-06,3.814697e-06,3.814697e-06,1.455192e-11,...,3.094916,3.985346,0.019056,0.027109,35.608988,45.915684,44.322680,54.664888,17.019060,17.497774
1,433,79599,0.0,0.0,0.0,9.145198,3.814697e-06,3.814697e-06,3.814697e-06,1.455192e-11,...,3.957160,5.488608,0.031845,0.050813,29.736341,35.174565,33.049206,41.584252,2.814765,2.206377
2,433,79600,0.0,0.0,0.0,8.394584,3.814697e-06,3.814697e-06,3.814697e-06,1.455192e-11,...,3.958647,5.488975,0.031858,0.050815,29.605743,35.064227,32.901368,41.743263,2.858828,2.370253
3,433,79601,0.0,0.0,0.0,9.903443,3.814697e-06,3.814697e-06,3.814697e-06,1.455192e-11,...,3.959931,5.488996,0.031868,0.050813,29.483046,34.960692,32.760495,41.907637,2.987122,2.620706
4,433,80317,0.0,0.0,0.0,12.290978,3.814697e-06,3.814697e-06,3.814697e-06,1.455192e-11,...,3.984358,5.535628,0.032332,0.051720,29.563491,35.014283,32.893768,40.804412,2.696757,2.148977
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1048795,502,190496,0.0,0.0,0.0,10.408626,5.199946e-07,5.199946e-07,5.199946e-07,2.703944e-13,...,4.584902,4.324970,0.069880,0.058797,11.916375,10.594810,14.326549,10.977249,1.524762,10.405138
1048796,502,190507,0.0,0.0,0.0,6.647283,5.199946e-07,5.199946e-07,5.199946e-07,2.703944e-13,...,4.652586,4.273955,0.092855,0.059402,8.154753,6.595453,9.899495,6.800735,2.649226,6.519210
1048797,502,190508,0.0,0.0,0.0,4.562102,5.199946e-07,5.199946e-07,5.199946e-07,2.703944e-13,...,4.653076,4.265913,0.095266,0.059457,7.889867,6.344289,9.552487,6.519202,2.787185,6.264990
1048798,502,190510,0.0,0.0,0.0,7.619576,5.199946e-07,5.199946e-07,5.199946e-07,2.703944e-13,...,4.660604,4.253587,0.100458,0.059811,7.433034,5.937171,8.902247,6.041523,3.281524,5.852358
